In [1]:
import pandas as pd
from collections import Counter
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import f1_score, accuracy_score 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nltk
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
data=pd.read_csv("/content/Constraint_Train.csv")
df=pd.read_csv("/content/english_test_with_labels.csv")

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import os
import gensim
import nltk 
from sklearn.linear_model import LogisticRegression

In [ ]:
df=pd.read_csv("/content/Constraint_Train.csv")

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for sentences in data['tweet']:
    print(sentences)

Streaming output truncated to the last 5000 lines.

https://t.co/FdaHtLE1Tc #workfromhomelife #cats #COVID19 https://t.co/KAumgbKowT
Our update is published. States reported record tests completed (852k) and cases (77k) for the second day in a row as well as 951 deaths. One significant caveat: the total cases include ~5k cases from a reporting backlog in Texas. https://t.co/Ky60XDlluD
Video of a man belonging to Tabligi Jamaat stripped naked and vandalizing the hospital where he's put under quarantine.
#Mauritius has high population density with high rates of non-communicable diseases &amp; many international travellers which meant it was at high-risk. But quick comprehensive action initiated in January &amp; previous experience with contact tracing paid off-@DrTedros #COVID19
Little chance we will see a coronavirus vaccine before November @AmerMedicalAssn says. https://t.co/fMeO2Bn3MA
Of the 89 cases in the community 88 are linked to the cluster and one remains under investigation. Th

In [ ]:
tokens = [nltk.word_tokenize(sentences) for sentences in df['tweet']]
train_tokens=[nltk.word_tokenize(sentences) for sentences in data['tweet']]

In [ ]:
model = gensim.models.Word2Vec(tokens, size=300, min_count=1, workers=4)
print("\n Training the word2vec model...\n")
model.save("word2vec.model")
model = gensim.models.Word2Vec.load("word2vec.model")
model.train(df.tweet, total_examples=len(df.tweet), epochs=4)
model.save("word2vecbody.model")
model.save("word2vecbody.model.wv.vectors.npy")
import numpy as np
np.load('word2vecbody.model.wv.vectors.npy',allow_pickle=True)


 Training the word2vec model...



In [ ]:
df.columns
data.columns

Index(['id', 'tweet', 'label'], dtype='object')

In [ ]:
train = []
test=[]
#getting only the first 4 columns of the file 
for sentences in df[data.columns[0:3]].values:
    train.extend(sentences)
for sentences in df[df.columns[0:3]].values:
    test.extend(sentences)
#print (train)

#Create an array of tokens using nltk
#for sentences in train:
#    tokens = tokens.append(nltk.word_tokenize(str(sentences)))
train_tokens = [nltk.word_tokenize(str(sentences)) for sentences in train]
tokens = [nltk.word_tokenize(str(sentences)) for sentences in test]

In [ ]:
model = gensim.models.Word2Vec(train_tokens, size=300, min_count=1, workers=4)
modeltest= gensim.models.Word2Vec(tokens, size=300, min_count=1, workers=4)
print("\n Training the word2vec model...\n")
#model = gensim.models.Word2Vec.load("word.model")
model.train(train_tokens, total_examples=len(train_tokens), epochs=4)
modeltest.train(tokens, total_examples=len(tokens), epochs=4)
model.save("word.model")
model.save("word.modeltest")


 Training the word2vec model...



In [ ]:
##from sklearn.model_selection import train_test_split
#X_headline_train, X_headline_test, y_headline_train, y_headline_test = train_test_split(X_headline,Y, test_size = 0.25, random_state=420)
#X_train, X_test, y_train, y_test = train_test_split(model.trainables.syn1neg[:len(df.label)],df['label'], test_size = 0.25, random_state=420)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['tweet'] = df['tweet'].apply(cleanText)
data['tweet'] = data['tweet'].apply(cleanText)

In [ ]:

import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['tweet']), tags=[r.label]), axis=1)
test_tagged = df.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['tweet']), tags=[r.label]), axis=1)

In [ ]:

train_tagged.values[3]

TaggedDocument(words=['indiafightscorona', 'we', 'have', '1524', 'covid', 'testing', 'laboratories', 'in', 'india', 'and', 'as', 'on', '25th', 'august', '2020', '36827520', 'tests', 'have', 'been', 'done', 'profbhargava', 'dg', 'icmrdelhi', 'staysafe', 'indiawillwin', 'url'], tags=['real'])

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 6420/6420 [00:00<00:00, 884926.61it/s]


In [ ]:
for epoch in range(45):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 6420/6420 [00:00<00:00, 1977631.59it/s]


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train_dbow, X_train_dbow = vec_for_learning(model_dbow, train_tagged)
y_test_dbow, X_test_dbow = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train_dbow, y_train_dbow)
y_pred_dbow = logreg.predict(X_test_dbow)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test_dbow, y_pred_dbow))
print('Testing F1 score: {}'.format(f1_score(y_test_dbow, y_pred_dbow, average='weighted')))

Testing accuracy 0.638006230529595
Testing F1 score: 0.6370468124971103


In [ ]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 6420/6420 [00:00<00:00, 592384.54it/s]


In [ ]:
for epoch in range(10):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|██████████| 6420/6420 [00:00<00:00, 1527104.39it/s]


In [ ]:
y_train_dm, X_train_dm = vec_for_learning(model_dmm, train_tagged)
y_test_dm, X_test_dm = vec_for_learning(model_dmm, test_tagged)
logreg.fit(X_train_dm, y_train_dm)
y_pred_dm = logreg.predict(X_test_dm)
print('Testing accuracy %s' % accuracy_score(y_test_dm, y_pred_dm))
print('Testing F1 score: {}'.format(f1_score(y_test_dm, y_pred_dm, average='weighted')))

Testing accuracy 0.9637071651090343
Testing F1 score: 0.9637104569956715


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


In [ ]:
!pip install testfixtures
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

     |████████████████████████████████| 102kB 3.9MB/s 


In [ ]:
def get_vectors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train, X_train = get_vectors(new_model, train_tagged)
y_test, X_test = get_vectors(new_model, test_tagged)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.8925233644859814
Testing F1 score: 0.8925135784509282


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
